# <center><font color="purple">**Import CSV**</font></center>

In [ ]:
# Memanggil library yg diperlukan
import pandas as pd
# Menyimpan path dari file data ke dalam variabel dataset
dataset = "/content/dataset.csv"
# Membaca File CSV ke DataFrame
df = pd.read_csv(dataset, sep=',', encoding='latin-1', on_bad_lines='skip')
# Melihat sedikit bagian atas data
df.head()

,conversation_id_str,created_at,full_text,username;;;;;;;;;;;;
0,1710806470810630000,Sat Oct 07 23:59:52 +0000 2023,@rgone360 Kena beli sorg lg wing back january ...,luqmanha14;;;;;;;;;;;;
1,1710652778232920000,Sat Oct 07 23:59:52 +0000 2023,@yogi_bintoro @yo2thok Kemungkinan dapet pas d...,AsySyaiful01;;;;;;;;;;;;
2,1710628935221080000,Sat Oct 07 23:59:50 +0000 2023,@kompascom Bangsaatt .... Hamas itu bukan tero...,haenesarajawane;;;;;;;;;;;;
3,1710807167656470000,Sat Oct 07 23:59:48 +0000 2023,Mana yg sokong Ukraine tempoh hari? Nahhh seka...,iammco;;;;;;;;;;;;
4,1710647444512570000,Sat Oct 07 23:59:46 +0000 2023,@tempodotco AS dukung israel ..Rusia dukung Pa...,CaprionDeni;;;;;;;;;;;;


# <center><font color="purple">**Data Pre-processing**</font></center>

In [ ]:
# Mengambil variabel penting untuk dianalisis
df_baru = df[['username;;;;;;;;;;;;','full_text']]
# Melihat sedikit bagian atas data
df_baru.head()

,username;;;;;;;;;;;;,full_text
0,luqmanha14;;;;;;;;;;;;,@rgone360 Kena beli sorg lg wing back january ...
1,AsySyaiful01;;;;;;;;;;;;,@yogi_bintoro @yo2thok Kemungkinan dapet pas d...
2,haenesarajawane;;;;;;;;;;;;,@kompascom Bangsaatt .... Hamas itu bukan tero...
3,iammco;;;;;;;;;;;;,Mana yg sokong Ukraine tempoh hari? Nahhh seka...
4,CaprionDeni;;;;;;;;;;;;,@tempodotco AS dukung israel ..Rusia dukung Pa...


In [ ]:
# Menghilangkan tanda titik koma pada kolom username
df_baru['username;;;;;;;;;;;;'] = df_baru['username;;;;;;;;;;;;'].str.replace(';', '')
# Mengganti nama kolom secara langsung
df_baru.columns = ['username', 'full_text']
# Melihat sedikit bagian atas data
df_baru.head()

<ipython-input-3-4976611e290f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_baru['username;;;;;;;;;;;;'] = df_baru['username;;;;;;;;;;;;'].str.replace(';', '')


,username,full_text
0,luqmanha14,@rgone360 Kena beli sorg lg wing back january ...
1,AsySyaiful01,@yogi_bintoro @yo2thok Kemungkinan dapet pas d...
2,haenesarajawane,@kompascom Bangsaatt .... Hamas itu bukan tero...
3,iammco,Mana yg sokong Ukraine tempoh hari? Nahhh seka...
4,CaprionDeni,@tempodotco AS dukung israel ..Rusia dukung Pa...


In [ ]:
# Melihat informasi data sebelum missing value dan duplikat dihapus
print('jumlah baris data sebelum dibershikan:', df_baru.shape)

# Membersihkan baris-baris yang mengandung missing value (Nan atau Null)
df_baru = df_baru.dropna()
# Menghapus data duplikat
df_baru = df_baru.drop_duplicates()

# Melihat informasi data setelah missing value dan duplikat dihapus
print('jumlah baris data setelah dibershikan:', df_baru.shape)

jumlah baris data sebelum dibershikan: (108966, 2)
jumlah baris data setelah dibershikan: (96946, 2)


In [ ]:
# Data untuk membangun graf
df_DK = df_baru
df_DK.head()

,username,full_text
0,luqmanha14,@rgone360 Kena beli sorg lg wing back january ...
1,AsySyaiful01,@yogi_bintoro @yo2thok Kemungkinan dapet pas d...
2,haenesarajawane,@kompascom Bangsaatt .... Hamas itu bukan tero...
3,iammco,Mana yg sokong Ukraine tempoh hari? Nahhh seka...
4,CaprionDeni,@tempodotco AS dukung israel ..Rusia dukung Pa...


# <center><font color="purple">**Membangun Graf**</font></center>

In [ ]:
# Mengimport library yang diperlukan
import re
import networkx as nx

# Membuat fungsi untuk mengambil pengguna-pengguna yang di-mention
def extract_mentions(text): # pendefinisian fungsi
  mention = re.findall(r'@(\w{4,15})', text) # menggunakan regular expression (regex) untuk menemukan mentions
  return mention # mengembalikan daftar mention

# Membuat graf terarah
G = nx.DiGraph()

# Mengiterasi baris pada DataFrame
for _, row in df_DK.iterrows(): # mengiterasi baris pada dataFrame
  tweet_text = row['full_text'] # mengambil teks tweet
  mentions = extract_mentions(tweet_text) # menemukan mentions

  # Menambahkan simpul dan sisi ke graf
  for mention in mentions: # mengiterasi setiap mention yang ditemukan dalam tweet
    if mention: # memastikan mention tidak kosong
      G.add_node(row['username']) # menambahkan pengguna yang memposting tweet sebagai simpul dalam graf
      G.add_node(mention) # menambahkan pengguna yang di-mention sebagai simpul dalam graf
      if mention != row['username']: # memastikan bahwa pengguna yang memposting tweet dan pengguna yang di-mention bukan orang yang sama
        if G.has_edge(row['username'], mention): # memeriksa apakah sudah ada sisi antara pengguna yang memposting tweet dan pengguna yang di-mention
          G[row['username']][mention]['weight']+=1 # jika sisi sudah ada, maka meningkatkan berat (weight) dari sisi tersebut sebesar 1
        else: # menambahkan pilihan lain
          G.add_edge(row['username'], mention, weight=1) # jika sisi belum ada, maka menambahkan sisi baru dengan berat 1

In [ ]:
# Melihat jumlah simpul yang terbentuk dari graf
print('jumlah simpul graf:', len(G.nodes))
# Melihat jumlah sisi yang terbentuk dari graf
print('jumlah sisi graf:', len(G.edges))

jumlah simpul graf: 54104
jumlah sisi graf: 71130


In [ ]:
'''
# Menyimpan graf ke dalam format gexf file
gexf_file = "graf.gexf"#
nx.write_gexf(G, gexf_file)

# Mengunduh data graf
from google.colab import files
files.download('graf.gexf')
'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>